In [1]:

!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

import os
os.environ['PATH'] += ":/root/.cargo/bin"

!cargo --version

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-04-03, rust version 1.86.0 (05f9846f8 2025-03-31)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
 27.1 MiB /  27.1 MiB (100 %)  25.2 MiB/s in  1s
info: downloading component 'rustc'
 72.8 MiB /  72.8 MiB (100 %)  35.8 MiB/s in  1s
info: downloading component 'rustfmt'
info: installing component 'cargo'
  8.8 MiB /   8.8 MiB (100 %)   7.4 MiB/s in  1s
info: installing component 'clippy'
info: installing component 'rust-docs'
 21.2 MiB /  21.2 MiB (100 %)   2.6 MiB/s in  8s
info: installing component 'rust-std'
 27.1 MiB /  27.1 MiB (100 %)   6.1 MiB/s in  4s
info: installing component 'rustc'
 72.8 MiB /  72.8 MiB (100 %)   7.6 MiB/s in  9s
info: installing component 'rustfmt'
i

In [2]:
!git clone https://github.com/edgenai/llama_cpp-rs.git

Cloning into 'llama_cpp-rs'...
remote: Enumerating objects: 1886, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 1886 (delta 90), reused 80 (delta 80), pack-reused 1701 (from 2)
Receiving objects: 100% (1886/1886), 405.43 KiB | 5.26 MiB/s, done.
Resolving deltas: 100% (977/977), done.


In [4]:
!cargo new llama_model_runner
%cd llama_model_runner

    Creating binary (application) `llama_model_runner` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html
/content/llama_model_runner


In [ ]:
[package]
name = "llama_model_runner"
version = "0.1.0"
edition = "2021"

[dependencies]
# اضبط المسار بما يتناسب مع موقع الكود على جهازك
llama_cpp = { path = "/content/llama_cpp-rs/crates/llama_cpp", features = ["default"] }

In [ ]:
use llama_cpp::{
    context::params::LlamaContextParams,
    model::params::LlamaModelParams,
    LlamaContext, LlamaModel, LlamaTokenizer,
};

fn main() -> Result<(), Box<dyn std::error::Error>> {
    // مسار لملف النموذج GGUF
    let model_path = "/content/mistral-7b-instruct-v0.2.Q2_K.gguf";

    // إعداد معلمات النموذج
    let model_params = LlamaModelParams::default()
        .n_gpu_layers(0); // ضبط عدد طبقات GPU المستخدمة (0 للـ CPU فقط)

    // تحميل النموذج
    let model = LlamaModel::load_from_file(model_path, model_params)?;

    // إعداد معلمات السياق
    let ctx_params = LlamaContextParams::default()
        .n_ctx(2048)  // حجم النافذة السياقية
        .n_threads(4); // عدد الخيوط

    // إنشاء السياق
    let mut ctx = LlamaContext::new(model, ctx_params)?;

    // استخدام النموذج للتوليد
    let prompt = "أكمل الجملة التالية: الذكاء الاصطناعي سوف";

    // المعلمات الأساسية للتوليد
    let temperature = 0.7;
    let top_p = 0.9;
    let max_tokens = 128;

    // توليد النص
    let output = ctx.generate(
        prompt,
        temperature,
        top_p,
        max_tokens,
        |_token| { true }, // استمر في التوليد
        |text| { print!("{}", text); }, // طباعة النص المولد
    )?;

    println!("\n\nالنص المولد الكامل: {}", output);

    Ok(())
}

In [5]:
%cd /content
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q2_K.gguf

/content
--2025-05-07 01:04:44--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q2_K.gguf
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/72/62/726219e98582d16c24a66629a4dec1b0761b91c918e15dea2625b4293c134a92/6d52a37032d25bc879691b0e870da12ffd5788e6f4dd2eea1b5d9a7020091f71?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.2.Q2_K.gguf%3B+filename%3D%22mistral-7b-instruct-v0.2.Q2_K.gguf%22%3B&Expires=1746583484&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NjU4MzQ4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzcyLzYyLzcyNjIxOWU5ODU4MmQxNmMyNGE2NjYyOWE0ZGVjMWIwNzYxYjkxYzkxOGUxNWRlYTI2MjViNDI5M2MxMzRhOTIvNmQ1MmEzNzAzMmQyNWJjODc5N

In [6]:
%%bash
# تثبيت متطلبات البناء
apt-get update && apt-get install -y build-essential cmake clang

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,894 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,266 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univer

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%bash
# تثبيت متطلبات البناء
apt-get update && apt-get install -y build-essential cmake clang

# تثبيت Rust
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
source $HOME/.cargo/env

In [7]:
%%bash

# إذا لم يكن المشروع موجودًا بالفعل
if [ ! -d "/content/llama_cpp-rs" ]; then
  git clone https://github.com/edgenai/llama_cpp-rs.git /content/llama_cpp-rs
fi

Cloning into '/content/llama_cpp-rs'...


In [8]:
%%writefile /content/llama_runner/src/main.rs
use llama_cpp::{
    context::params::LlamaContextParams,
    model::params::LlamaModelParams,
    LlamaContext, LlamaModel,
};

fn main() -> Result<(), Box<dyn std::error::Error>> {
    // طريق النموذج - تأكد من تحديثه إذا استخدمت نموذجًا مختلفًا
    let model_path = "/content/models/phi-2.Q4_K_M.gguf";

    println!("تحميل النموذج من: {}", model_path);

    // إعداد معلمات النموذج للـ CPU
    let model_params = LlamaModelParams::default()
        .n_gpu_layers(0)       // تعطيل GPU
        .use_mmap(true)        // استخدام mmap لتوفير الذاكرة
        .use_mlock(false);     // عدم قفل الذاكرة

    // تحميل النموذج
    let model = match LlamaModel::load_from_file(model_path, model_params) {
        Ok(model) => {
            println!("تم تحميل النموذج بنجاح!");
            model
        },
        Err(e) => {
            println!("خطأ في تحميل النموذج: {}", e);
            return Err(e.into());
        }
    };

    // تكوين معلمات السياق
    let ctx_params = LlamaContextParams::default()
        .n_ctx(512)            // حجم نافذة سياقية صغير للتوفير في الذاكرة
        .n_batch(64)           // حجم دفعة صغير
        .n_threads(2);         // يمكن زيادته حسب عدد وحدات المعالجة المركزية المتاحة في Colab

    // إنشاء السياق
    let mut ctx = match LlamaContext::new(model, ctx_params) {
        Ok(ctx) => {
            println!("تم إنشاء السياق بنجاح!");
            ctx
        },
        Err(e) => {
            println!("خطأ في إنشاء السياق: {}", e);
            return Err(e.into());
        }
    };

    // المطالبة - استخدم مطالبة بسيطة لاختبار النموذج
    let prompt = "اكتب جملة قصيرة عن الذكاء الاصطناعي:";
    println!("المطالبة: {}", prompt);

    // معلمات التوليد
    let temperature = 0.7;
    let top_p = 0.9;
    let max_tokens = 50;  // عدد الرموز المراد توليدها

    println!("بدء التوليد...");

    // توليد النص
    let output = ctx.generate(
        prompt,
        temperature,
        top_p,
        max_tokens,
        |_token| { true },  // استمر في التوليد
        |text| { print!("{}", text); }, // طباعة مباشرة للنص المولد
    )?;

    println!("\n\nالنص المولد الكامل: {}", output);

    Ok(())
}

Writing /content/llama_runner/src/main.rs


In [9]:
%%bash
cd /content/llama_runner
cargo build --release
./target/release/llama_runner

    Updating crates.io index
     Locking 84 packages to latest compatible versions
  Downloaded autocfg v1.4.0
  Downloaded convert_case v0.4.0
  Downloaded cfg-if v1.0.0
  Downloaded cexpr v0.6.0
  Downloaded link-cplusplus v1.0.10
  Downloaded which v4.4.2
  Downloaded thiserror v1.0.69
  Downloaded unicode-ident v1.0.18
  Downloaded tracing-core v0.1.33
  Downloaded tracing v0.1.41
  Downloaded prettyplease v0.2.32
  Downloaded minimal-lexical v0.2.1
  Downloaded nom v7.1.3
  Downloaded itertools v0.12.1
  Downloaded aho-corasick v1.1.3
  Downloaded futures-util v0.3.31
  Downloaded bindgen v0.69.5
  Downloaded memchr v2.7.4
  Downloaded cc v1.2.21
  Downloaded regex v1.11.1
  Downloaded once_cell v1.21.3
  Downloaded syn v2.0.101
  Downloaded log v0.4.27
  Downloaded futures v0.3.31
  Downloaded regex-syntax v0.8.5
  Downloaded rustix v0.38.44
  Downloaded derive_more v0.99.20
  Downloaded tracing-attributes v0.1.28
  Downloaded thiserror-impl v1.0.69
  Downloaded quote v1.0.40
  

CalledProcessError: Command 'b'cd /content/llama_runner\ncargo build --release\n./target/release/llama_runner\n'' returned non-zero exit status 127.

In [10]:
%%bash
# حذف المجلد الحالي إذا كان موجوداً
rm -rf /content/llama_cpp-rs

# استنساخ المستودع مع المستودعات الفرعية
git clone --recursive https://github.com/edgenai/llama_cpp-rs.git /content/llama_cpp-rs

# إذا كان المستودع موجودًا بالفعل ولكن بدون تحميل المستودعات الفرعية
cd /content/llama_cpp-rs
git submodule update --init --recursive

Submodule path 'crates/llama_cpp_sys/thirdparty/Vulkan-Headers': checked out '46dc0f6e514f5730784bb2cac2a7c731636839e8'
Submodule path 'crates/llama_cpp_sys/thirdparty/llama.cpp': checked out '60cdf40cc32f0ad4cb11e0ca8fd38f3b93d8d640'
Submodule path 'crates/llama_cpp_sys/thirdparty/llama.cpp/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'


Cloning into '/content/llama_cpp-rs'...
Submodule 'crates/llama_cpp_sys/thirdparty/Vulkan-Headers' (https://github.com/KhronosGroup/Vulkan-Headers.git) registered for path 'crates/llama_cpp_sys/thirdparty/Vulkan-Headers'
Submodule 'crates/llama_cpp_sys/thirdparty/llama.cpp' (https://github.com/ggerganov/llama.cpp.git) registered for path 'crates/llama_cpp_sys/thirdparty/llama.cpp'
Cloning into '/content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/Vulkan-Headers'...
Cloning into '/content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/llama.cpp'...
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'crates/llama_cpp_sys/thirdparty/llama.cpp/kompute'
Cloning into '/content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/llama.cpp/kompute'...


In [11]:
%%bash
# التحقق من وجود ملف llama.h
ls -la /content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/llama.cpp/llama.h

# إذا لم يكن الملف موجودًا، فتأكد من وجود المجلد
ls -la /content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/

-rw-r--r-- 1 root root 47240 May  7 01:12 /content/llama_cpp-rs/crates/llama_cpp_sys/thirdparty/llama.cpp/llama.h
total 16
drwxr-xr-x  4 root root 4096 May  7 01:11 .
drwxr-xr-x  5 root root 4096 May  7 01:11 ..
drwxr-xr-x 22 root root 4096 May  7 01:12 llama.cpp
drwxr-xr-x  8 root root 4096 May  7 01:12 Vulkan-Headers


In [12]:
%%bash
# إنشاء مجلد للمشروع الخاص بك (أو إعادة إنشائه)
rm -rf /content/llama_runner
mkdir -p /content/llama_runner
cd /content/llama_runner

# إنشاء مشروع Rust جديد
cargo init --bin

    Creating binary (application) package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [13]:
%%writefile /content/llama_runner/Cargo.toml
[package]
name = "llama_runner"
version = "0.1.0"
edition = "2021"

[dependencies]
# استخدام مكتبة llama_cpp المحلية مع تفعيل الميزات الأساسية فقط
llama_cpp = { path = "/content/llama_cpp-rs/crates/llama_cpp", default-features = false, features = ["native"] }

Overwriting /content/llama_runner/Cargo.toml


In [14]:
%%writefile /content/llama_runner/src/main.rs
// كود بسيط للتحقق من أن المكتبة تعمل بشكل صحيح
fn main() {
    println!("بدء تشغيل تطبيق llama_runner");

    // طباعة معلومات عن المكتبة
    println!("محاولة استخدام مكتبة llama_cpp...");

    // اختبار بسيط لمعرفة ما إذا كانت المكتبة متاحة
    match llama_cpp::LlamaModel::list_available_models() {
        Ok(models) => {
            println!("النماذج المتاحة: {:?}", models);
        },
        Err(e) => {
            println!("خطأ في استدعاء المكتبة: {}", e);
        }
    }

    println!("انتهاء التنفيذ");
}

Overwriting /content/llama_runner/src/main.rs


In [15]:
%%bash
cd /content/llama_runner
# بناء المشروع مع مستوى تشخيص أعلى
RUST_LOG=debug cargo build -v

    Updating crates.io index
     Locking 84 packages to latest compatible versions
   Compiling proc-macro2 v1.0.95
   Compiling unicode-ident v1.0.18
     Running `/root/.rustup/toolchains/stable-x86_64-unknown-linux-gnu/bin/rustc --crate-name build_script_build --edition=2021 /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/proc-macro2-1.0.95/build.rs --error-format=json --json=diagnostic-rendered-ansi,artifacts,future-incompat --crate-type bin --emit=dep-info,link -C embed-bitcode=no --cfg 'feature="default"' --cfg 'feature="proc-macro"' --check-cfg 'cfg(docsrs,test)' --check-cfg 'cfg(feature, values("default", "nightly", "proc-macro", "span-locations"))' -C metadata=b3397f274477117c -C extra-filename=-623611f8ef0176fe --out-dir /content/llama_runner/target/debug/build/proc-macro2-623611f8ef0176fe -L dependency=/content/llama_runner/target/debug/deps --cap-lints allow`
     Running `/root/.rustup/toolchains/stable-x86_64-unknown-linux-gnu/bin/rustc --crate-name unicode_id

CalledProcessError: Command 'b'cd /content/llama_runner\n# \xd8\xa8\xd9\x86\xd8\xa7\xd8\xa1 \xd8\xa7\xd9\x84\xd9\x85\xd8\xb4\xd8\xb1\xd9\x88\xd8\xb9 \xd9\x85\xd8\xb9 \xd9\x85\xd8\xb3\xd8\xaa\xd9\x88\xd9\x89 \xd8\xaa\xd8\xb4\xd8\xae\xd9\x8a\xd8\xb5 \xd8\xa3\xd8\xb9\xd9\x84\xd9\x89\nRUST_LOG=debug cargo build -v\n'' returned non-zero exit status 101.

In [16]:
%%writefile /content/llama_runner/Cargo.toml
[package]
name = "llama_runner"
version = "0.1.0"
edition = "2021"

[dependencies]
# استخدام الإصدار المنشور من crates.io بدلاً من المستودع المحلي
llama_cpp = "0.3.2"

Overwriting /content/llama_runner/Cargo.toml


In [ ]:
!apt-get update && apt-get install -y build-essential cmake

In [ ]:
%%bash
cd /content/llama_runner
# بناء المشروع مع مستوى تشخيص أعلى
RUST_LOG=debug cargo build -v

In [17]:
%%bash
cd /content/llama_runner
cargo build

    Updating crates.io index
     Locking 2 packages to latest compatible versions
      Adding llama_cpp v0.3.2
      Adding llama_cpp_sys v0.3.2
  Downloaded llama_cpp v0.3.2
  Downloaded llama_cpp_sys v0.3.2
   Compiling llama_cpp_sys v0.3.2
   Compiling llama_cpp v0.3.2
   Compiling llama_runner v0.1.0 (/content/llama_runner)
error[E0599]: no function or associated item named `list_available_models` found for struct `LlamaModel` in the current scope
   --> src/main.rs:9:34
    |
9   |     match llama_cpp::LlamaModel::list_available_models() {
    |                                  ^^^^^^^^^^^^^^^^^^^^^ function or associated item not found in `LlamaModel`
    |
note: if you're trying to build a new `LlamaModel`, consider using `LlamaModel::load_from_file` which returns `Result<LlamaModel, LlamaLoadError>`
   --> /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/llama_cpp-0.3.2/src/model/mod.rs:161:5
    |
161 | /     pub fn load_from_file(
162 | |         file_path: impl A

CalledProcessError: Command 'b'cd /content/llama_runner\ncargo build\n'' returned non-zero exit status 101.

In [1]:
%cd /content/llama_cpp-rs/crates/llama_cpp
!cargo build

/content/llama_cpp-rs/crates/llama_cpp
/bin/bash: line 1: cargo: command not found


In [20]:
!apt-get install clang

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
clang is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
import os
os.environ['PATH'] += ":/root/.cargo/bin"

!cargo --version

cargo 1.86.0 (adf9b6ad1 2025-02-28)


In [3]:
! cargo run --release

error: a bin target must be available for `cargo run`


In [5]:
%cd llama_cpp-rs

[Errno 2] No such file or directory: 'llama_cpp-rs'
/content/llama_cpp-rs/crates/llama_cpp


In [6]:
!cargo build --release

   Compiling proc-macro2 v1.0.79
   Compiling unicode-ident v1.0.12
   Compiling libc v0.2.153
   Compiling quote v1.0.35
   Compiling glob v0.3.1
   Compiling syn v2.0.53
   Compiling clang-sys v1.7.0
   Compiling jobserver v0.1.28
   Compiling rustix v0.38.32
   Compiling prettyplease v0.2.16
   Compiling cc v1.0.90
   Compiling autocfg v1.1.0
   Compiling minimal-lexical v0.2.1
   Compiling either v1.10.0
   Compiling regex-syntax v0.8.2
   Compiling memchr v2.7.1
   Compiling cfg-if v1.0.0
   Compiling linux-raw-sys v0.4.13
   Compiling bitflags v2.5.0
   Compiling nom v7.1.3
   Compiling libloading v0.8.3
   Compiling regex-automata v0.4.6
   Compiling slab v0.4.9
   Compiling pin-project-lite v0.2.13
   Compiling home v0.5.9
   Compiling bindgen v0.69.4
   Compiling which v4.4.2
   Compiling regex v1.10.3
   Compiling cexpr v0.6.0
   Compiling itertools v0.12.1
   Compiling link-cplusplus v1.0.9
   Compiling futures-sink v0.3.30
   Compiling lazycell v1.3.0
   Compiling shlex v1.